## import the libraries to use

In [1]:
# This is for webscrapping
from bs4 import BeautifulSoup
import csv
import io

# Pandas
import pandas as pd

# REGEX
import re

# For get the date and time

from datetime import date
from datetime import datetime

# For create the engine and works with db's
import requests
from sqlalchemy.types import Integer, Text, String, DateTime
from sqlalchemy_utils import create_database, database_exists, drop_database
from sqlalchemy import create_engine
#import psycopg2
import sqlite3

## functions

In [2]:
# With this function I make the webscrapping I need to extract the data from the tarifaluzahora website
def scrapping (tarifa, day = str(date.today())):
    
    url = 'https://tarifaluzhora.es/?tarifa=' + tarifa
    page = requests.get(url)
    soup = BeautifulSoup(page.text, "html.parser")
    
    price_ = soup.findAll("span", {"itemprop": "price"})
    hours_ = soup.findAll("span", {"itemprop": "description"})
    
    price_hour_ = [price.get_text() for price in price_]
    schedule_ = [time.get_text() for time in hours_]
    
    df = pd.DataFrame.from_dict({'precio':price_hour_,'horario':schedule_})
    
    df['hora'] = [int(x[:2]) for x in df['horario']]
    df['tarifa'] = tarifa
    df['precio'] =  [re.sub(r'/[k][W][h]','', str(x)) for x in df['precio']]
    df['horario'] =  [re.sub(r'[:]','', str(x)) for x in df['horario']]
    
    return df

## extract fares from scrapping function

In [3]:
df = scrapping('coche_electrico')

In [4]:
df = df.groupby("precio").min().reset_index()

In [11]:
precio = df.precio[0]
precio

'0.03203 €'

In [6]:
hora = df.hora[0]
hora

1

In [7]:
tarifa = df.tarifa[0]
tarifa

'coche_electrico'